In [5]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
import pydicom as dicom
import scipy.misc
from glob import glob
import os

#PLEASE RUN THIS CELL 
import requests
import os

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from skimage.io import imread
from scipy.misc import imresize
from sklearn.model_selection import train_test_split


#from keras.models import Sequential, Model, load_model
#from keras.layers import *
#from keras.utils import np_utils, to_categorical
#from keras.losses import binary_crossentropy
#from keras import backend as K,objectives
#from keras.losses import mse, binary_crossentropy
#from keras.layers.advanced_activations import LeakyReLU
#from keras.optimizers import Adam, RMSprop
#from keras.initializers import RandomNormal
#from keras.preprocessing import image

import random
#import cv2

from PIL import Image

from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [6]:
DATA_FOLDER = '../raw_data'
ATTR_FOLDER = '../attributes'

datasets = ['mass']
subsets = ['train', 'test']

OUTPUT_FOLDER = '../clean_data'
IM_OUTPUT = 'images'
ROI_OUTPUT = 'masks'
CROP_OUTPUT = 'crops'
MASK_BLACKBOX_OUTPUT = 'mask_blackbox'

IM_DIMS = (512, 512)

In [ ]:
resnet_dict = {
    'path': [],
    'class': [],
}

for dataset in datasets:
    for subset in subsets:
        df_attrs = pd.read_csv(f'{ATTR_FOLDER}/{dataset}_case_description_{subset}_set.csv')

        for i in range(len(df_attrs)):
            # get paths
            im_path_lst = df_attrs['image file path'][i].split('/')
            roi_path_lst = df_attrs['ROI mask file path'][i].split('/')
            crop_path_lst = df_attrs['cropped image file path'][i].split('/')

            im_path = glob(
                f'{DATA_FOLDER}/{dataset}_{subset}ing_data/CBIS-DDSM/{im_path_lst[0]}/*/*/*'
            )
    
            crop_paths = glob(
                f'{DATA_FOLDER}/{dataset}_{subset}ing_ROI_data/CBIS-DDSM/{crop_path_lst[0]}/*/*/*'
            )

            # if images found for both types save to new folders
            if im_path and crop_paths:
                
                # original image
                try:
                    im_pxl = dicom.dcmread(im_path[0]).pixel_array
                except ValueError:
                    print('Error: ')
                    print(im_path[0])
                
                im_pxl = cv2.resize(im_pxl, IM_DIMS)
                scipy.misc.imsave(f'{OUTPUT_FOLDER}/{dataset}/{subset}/{IM_OUTPUT}/{i}.tif', im_pxl)

                # cropped and masks
                crop_path1, crop_path2 = crop_paths
                try: 
                    crop_pxl_1 = dicom.dcmread(crop_path1).pixel_array
                    crop_pxl_2 = dicom.dcmread(crop_path2).pixel_array
                except ValueError:
                    print('Error: ')
                    print(crop_path1)

                # set temporary values
                bigger = crop_pxl_1
                smaller = crop_pxl_2
                if os.path.getsize(crop_path1) < os.path.getsize(crop_path2):
                    bigger = crop_pxl_2
                    smaller = crop_pxl_1

                smaller = cv2.resize(smaller, IM_DIMS)
                bigger = cv2.resize(bigger, IM_DIMS)

                scipy.misc.imsave(f'{OUTPUT_FOLDER}/{dataset}/{subset}/{CROP_OUTPUT}/{i}.tif', smaller)
                scipy.misc.imsave(f'{OUTPUT_FOLDER}/{dataset}/{subset}/{ROI_OUTPUT}/{i}.tif', bigger)

                # save blackbox images
                if df_attrs.pathology[i] != 'MALIGNANT':
                    clss = 0
                    scipy.misc.imsave(
                        f'{OUTPUT_FOLDER}/{dataset}/{subset}/{MASK_BLACKBOX_OUTPUT}/{i}.tif', 
                        np.zeros(IM_DIMS),
                    )
                else:
                    clss = 1
                    scipy.misc.imsave(f'{OUTPUT_FOLDER}/{dataset}/{subset}/{MASK_BLACKBOX_OUTPUT}/{i}.tif', bigger)

                # add to DataFrame
                resnet_dict['path'].append(f'{i}.tif')
                resnet_dict['class'].append(clss)

        pd.DataFrame(resnet_dict).to_csv(f'{OUTPUT_FOLDER}/{dataset}/{subset}/resnet_data.csv')